Once your ee.Image is exported to GEE, run this to vectorize the ee.Image and export it as shapefile to Google Drive. 

FCAT format requires shapefile with unique codes identifying the treatment timing

In [11]:
import ee
import os
from src.utils.yml_params import get_export_params
from src.utils import runs
yml_file = os.path.join(os.getcwd(),'config.yml')
CRS,EXPORT_SCALE = get_export_params(yml_file)

ee.Initialize(project='mas-gee')
%load_ext autoreload
%autoreload 2

# In development phase
tag = 'vectorize_export'
ee.data.setWorkloadTag(tag)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INPUT YOUR EE TREATMENT ASSET PATH BELOW THEN RUN ALL

In [12]:
ee_treat_img_path = "projects/mas-gee/assets/treatment_scenarios/RunID35/CC_RFFC_2m_20231020"
RUNID=ee_treat_img_path.split('/')[4]
print(RUNID)

RunID35


In [13]:
run_id = os.path.dirname(ee_treat_img_path).split('/')[-1]
output_shapefile = f'{run_id}_TreatmentsSHP_{os.path.basename(ee_treat_img_path)}'
print(f'Output Shapefile Path: {output_shapefile}')
desc = f'Vectorize_{os.path.basename(output_shapefile)}'
print(desc)

img = ee.Image(ee_treat_img_path)
folder = f'MAS_FCATtreatments'

# vectorize the image
vectors = ee.Image(img).reduceToVectors(**{
    # 'reducer':ee.Reducer.firstNonNull(), 
    # geometry, 
    'scale':30, 
    'geometryType':'polygon', 
    'eightConnected':True, 
    'labelProperty':'tx_code', # FCAT required field name
    'crs':CRS, 
    # crsTransform, 
    'bestEffort':True, 
    'maxPixels':1e13, 
    'tileScale':4, 
    'geometryInNativeProjection':True})

# other FCAT required fields 
vectors = vectors.map(lambda f: f.set('kcp_exists',1))
# print(vectors.first().getInfo()['properties'])

Output Shapefile Path: RunID35_TreatmentsSHP_CC_RFFC_2m_20231020
Vectorize_RunID35_TreatmentsSHP_CC_RFFC_2m_20231020


In [14]:
lookupIn = ee.List(runs.key[RUNID]["vecCodeLookUpList"])
lookupOut = ee.List(runs.key[RUNID]["vecTypeLookUpList"])
zipped_list = lookupIn.zip(lookupOut)
# print(zipped_list.getInfo())
def fcFieldRemapper (l):
    in_val = ee.List(l).get(0)
    out_val = ee.List(l).get(1)
    filteredFeats = (
        (vectors.filter(ee.Filter.eq('tx_code',in_val)) # get all Features w a given tx_code
         .map( # for each Feature in that filter result, set tx_code and tx_type props
            lambda f: ee.Feature(f).set('tx_code',in_val,'tx_type',out_val)
            .copyProperties(vectors))
        ))
    return ee.FeatureCollection(filteredFeats) 

final_output = ee.FeatureCollection(zipped_list.map(fcFieldRemapper)).flatten()
# print('test we are setting tx_type and code properly..\n',
#       final_output.first().getInfo()['properties'])

In [15]:
# export shapefile to Drive
task = ee.batch.Export.table.toDrive(**{
    'collection':final_output, 
    'description':desc, 
    'folder':folder, 
    'fileNamePrefix':output_shapefile, 
    'fileFormat':'SHP', 
    'selectors': ['count','kcp_exists','tx_code','tx_type'], 
    # 'maxVertices'
    })

task.start()
print(f'Export started for {folder}/{output_shapefile}')

Export started for MAS_FCATtreatments/RunID35_TreatmentsSHP_CC_RFFC_2m_20231020
